In [1]:
import casadi as ca
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
S = 2170.0
CBAR = 17.5
MASS = 5000
IYY = 4.1*10**6
TSTAT = 6.0*10**4
DTDV = -38.0
ZE = 2.0
CDCLS = 0.042
CLA = 0.085
CMA = -0.22
CMDE = -0.16
CMQ = -16.0
CMADOT= -6.0
CLADOT = 0
RTOD = 57.29578
GD = 32.17

In [3]:
THTL = ca.SX.sym('th')
ELEV = ca.SX.sym('el')
XCG = ca.SX.sym('xcg')
LAND = ca.SX.sym('ld')

u = ca.vertcat(THTL,ELEV,XCG,LAND)

VT = ca.SX.sym('vt')
ALPHA = RTOD * ca.SX.sym('a')
THETA = ca.SX.sym('theta')
Q = ca.SX.sym('q')
H = ca.SX.sym('h')

x = ca.vertcat(VT,ALPHA,THETA,Q,H)

In [4]:
#PAGE 634 FOR ADC IN FORTRAN
#QBAR = ca.SX.sym('Qbar')
#MACH = ca.SX.sym('mach')
ALT = ca.SX.sym('alt')
DATA  = R0 = 2.377*10**-3
TFAC = 1.0 - 0.703*10**-5 * ALT
T = ca.if_else(ALT >= 35000, 390, 519*TFAC)
RHO = R0 * (TFAC**4.14)
AMACH = VT/(1.4*1716.3*T)**0.5
QBAR = 0.5*RHO*VT*VT
C_PS = 1715.0 * RHO * T


In [5]:
QS = QBAR * S
SALP = ca.sin(x[1])
CALP = ca.cos(x[1])
GAM = THETA - x[1]
SGAM = ca.sin(GAM)
CGAM = ca.cos(GAM)

In [6]:
land = ca.SX.sym('land')

LAND0 = (0.20,0.16,0.05,0.0,0.0)
LAND1 = (1.0,0.08,-0.20,0.02,-0.05)

coeff = ca.if_else(land, LAND0, LAND1)
CLO = coeff[0]
CDO = coeff[1]
CMO = coeff[2]
DCDG = coeff[3]
DCMG = coeff[4]


In [7]:
THR = (TSTAT+VT*DTDV) * ca.fmax(THTL,0)
CL = CLO + CLA*ALPHA
CM = DCMG + CMO + CMA*ALPHA + CMDE*ELEV + CL* (XCG-0.25)
CD = DCDG + CDO + CDCLS*CL*CL

In [8]:
xd1 = (THR*CALP*QS*CD)/MASS - GD * SGAM
xd2 = (-THR*SALP-QS*CL+MASS*(VT*Q+GD+CGAM)) / (MASS*VT+QS*CLADOT)
xd3 = Q
D = 0.5*CBAR*(CMQ*Q+CMADOT*xd2)/VT
xd4 = (QS*CBAR*(CM + D) + THR*ZE)/IYY
xd5 = VT*SGAM
xd6 = VT*CGAM

In [9]:
xd = ca.vertcat(xd1,xd2,xd3,xd4,xd5,xd6)


In [10]:
from casadi.tools.graph import dotgraph
from IPython.display import Image

def draw_graph(expr):
    return Image(dotgraph(expr).create_png())

In [ ]:
draw_graph(xd)